In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## conv

In [ ]:
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial)

def conv2d(x, W, strides=[1, 1, 1, 1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
def add_histogram(var, name):
    with tf.name_scope('summaries'):
        tf.summary.histogram(name, var)

In [ ]:
tf.reset_default_graph() 

# build computation graph
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(x, [-1, 28, 28, 1])

strides = [1, 1, 1, 1]

with tf.name_scope('conv_1') as scope:
    W_conv1 = weight_var([3, 3, 1, 32])
    b_conv1 = bias_var([32])
    h_conv1 = tf.nn.relu(conv2d(x_img, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    add_histogram(W_conv1, 'w_conv_1')
    add_histogram(b_conv1, 'b_conv1')

with tf.name_scope('conv_2') as scope:
    W_conv2 = weight_var([7, 7, 32, 64])
    b_conv2 = bias_var([64])
    
    add_histogram(W_conv2, 'w_conv_2')
    add_histogram(b_conv2, 'b_conv_2')
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

with tf.name_scope('fc_1') as scope:
    W_fc1 = weight_var([7*7*64, 1024])
    b_fc1 = bias_var([1024])
    
    add_histogram(W_fc1, 'w_fc1')
    add_histogram(b_fc1, 'b_fc1')
    
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    add_histogram(h_fc1, 'h_fc1')

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope('output') as scope:
    W_fc2 = weight_var([1024, 10])
    b_fc2 = bias_var([10])

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

In [105]:
summaries_dir = './summaries/histograms3/'

In [106]:
%%time
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sesh:
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sesh.graph)
    test_writer = tf.summary.FileWriter(summaries_dir + '/test')
    sesh.run(tf.global_variables_initializer())
    batch_size = 50
    
    test_xs, test_ys = mnist.test.images, mnist.test.labels
    
    for i in range(1000):
        batch = mnist.train.next_batch(batch_size)
        if i % 10 == 0: # record test set accuracy
            summary, acc = sesh.run([merged, accuracy], feed_dict={x: test_xs, y_: test_ys, keep_prob: 1.})
            test_writer.add_summary(summary, i)
        else:
            if i % 100 == 99: # record train set accuracy
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sesh.run([merged, train_step], 
                                      feed_dict={x: batch[0],
                                                 y_: batch[1],
                                                 keep_prob: 0.5},
                                      options=run_options,
                                      run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sesh.run([merged, train_step], feed_dict={x: batch[0],
                                                 y_: batch[1],
                                                 keep_prob: 0.5})
                train_writer.add_summary(summary, i)
test_writer.close()
train_writer.close()


Adding run metadata for 99
Adding run metadata for 199
Adding run metadata for 299
Adding run metadata for 399
Adding run metadata for 499
Adding run metadata for 599
Adding run metadata for 699
Adding run metadata for 799
Adding run metadata for 899
Adding run metadata for 999
CPU times: user 4min 23s, sys: 3.05 s, total: 4min 26s
Wall time: 3min 45s


In [13]:

conv1_layer = tf.layers.conv2d(
    inputs=x_img,
    filters = 16,
    kernel_size = (4, 4),
    strides=(2, 2),
    padding='same',
    kernel_initializer=tf.contrib.layers.xavier_initializer(),
    bias_initializer=tf.zeros_initializer(),
    kernel_regularizer=None,
    name = 'conv1',
    activation = tf.nn.elu
)

In [22]:
conv1_layer.consumers()

[]